<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/vector_stores/Neo4jVectorDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neo4j vector store

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [1]:
%pip install llama-index neo4j wget

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "sk-xxx"
openai.api_key = os.environ["OPENAI_API_KEY"]

## Initiate Neo4j vector wrapper
### docker 启动 neo4j
```shell
docker run -d --name testneo4j -p7474:7474 -p7687:7687  --env NEO4J_AUTH=neo4j/pleaseletmein neo4j:latest
```

In [8]:
from llama_index.vector_stores import Neo4jVectorStore

username = "neo4j"
password = "pleaseletmein"
url = "bolt://localhost:7687"
embed_dim = 1536

neo4j_vector = Neo4jVectorStore(username, password, url, embed_dim)

## Load documents, build the VectorStoreIndex

In [9]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from IPython.display import Markdown, display

Download Data

In [12]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

1289.57s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
1294.75s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
zsh:1: command not found: wget


In [13]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham").load_data()

In [14]:
from llama_index.storage.storage_context import StorageContext

storage_context = StorageContext.from_defaults(vector_store=neo4j_vector)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What happened at interleaf?")
display(Markdown(f"<b>{response}</b>"))

<b>At Interleaf, they added a scripting language inspired by Emacs and made it a dialect of Lisp. They were looking for a Lisp hacker to write things in this scripting language. The author of the text worked at Interleaf and mentioned that their Lisp was the thinnest icing on a giant C cake. The author also mentioned that they didn't know C and didn't want to learn it, so they never understood most of the software at Interleaf. Additionally, the author admitted to being a bad employee and spending much of their time working on a separate project called On Lisp.</b>

## Hybrid search

Hybrid search uses a combination of keyword and vector search
In order to use hybrid search, you need to set the `hybrid_search` to `True`

In [ ]:
neo4j_vector_hybrid = Neo4jVectorStore(
    username, password, url, embed_dim, hybrid_search=True
)

In [ ]:
storage_context = StorageContext.from_defaults(
    vector_store=neo4j_vector_hybrid
)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)
query_engine = index.as_query_engine()
response = query_engine.query("What happened at interleaf?")
display(Markdown(f"<b>{response}</b>"))

<b>At Interleaf, they added a scripting language inspired by Emacs and made it a dialect of Lisp. They were looking for a Lisp hacker to write things in this scripting language. The author of the essay worked at Interleaf but didn't understand most of the software because he didn't know C and didn't want to learn it. He also mentioned that their Lisp was the thinnest icing on a giant C cake. The author admits to being a bad employee and spending much of his time working on a contract to publish On Lisp.</b>

## Load existing vector index

In order to connect to an existing vector index, you need to define the `index_name` and `text_node_property` parameters:

- index_name: name of the existing vector index (default is `vector`)
- text_node_property: name of the property that containt the text value (default is `text`)

In [ ]:
index_name = "existing_index"
text_node_property = "text"
existing_vector = Neo4jVectorStore(
    username,
    password,
    url,
    embed_dim,
    index_name=index_name,
    text_node_property=text_node_property,
)

loaded_index = VectorStoreIndex.from_vector_store(existing_vector)

## Customizing responses

You can customize the retrieved information from the knowledge graph using the `retrieval_query` parameter.

The retrieval query must return the following four columns:

* text:str - The text of the returned document
* score:str - similarity score
* id:str - node id
* metadata: Dict - dictionary with additional metadata (must contain `_node_type` and `_node_content` keys)

In [ ]:
retrieval_query = (
    "RETURN 'Interleaf hired Tomaz' AS text, score, node.id AS id, "
    "{author: 'Tomaz', _node_type:node._node_type, _node_content:node._node_content} AS metadata"
)
neo4j_vector_retrieval = Neo4jVectorStore(
    username, password, url, embed_dim, retrieval_query=retrieval_query
)

In [ ]:
loaded_index = VectorStoreIndex.from_vector_store(
    neo4j_vector_retrieval
).as_query_engine()
response = loaded_index.query("What happened at interleaf?")
display(Markdown(f"<b>{response}</b>"))

<b>Interleaf hired Tomaz.</b>